In [1]:
#imports
using Pkg; for p in ("Knet","IterTools"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Statistics: mean
using IterTools: ncycle
using WordTokenizers
using Test
using Knet
using Random

In [2]:
train_file = "/home/burak/nn4nlp-code/data/ptb/train.txt"
test_file = "/home/burak/nn4nlp-code/data/ptb/valid.txt"

"/home/burak/nn4nlp-code/data/ptb/valid.txt"

In [3]:
wdict = Dict()
w2i(x)= get!(wdict,x,1+length(wdict))

w2i (generic function with 1 method)

In [4]:
@doc append!

```
append!(collection, collection2) -> collection.
```

Add the elements of `collection2` to the end of `collection`.

# Examples

```jldoctest
julia> append!([1],[2,3])
3-element Array{Int64,1}:
 1
 2
 3

julia> append!([1, 2, 3], [4, 5, 6])
6-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
```

Use [`push!`](@ref) to add individual items to `collection` which are not already themselves in another collection. The result of the preceding example is equivalent to `push!([1, 2, 3], 4, 5, 6)`.

```
append!(cb, datavec)
```

Push at most last `capacity` items.


In [4]:
function r(fname)
    sent_arr = []
  open(fname, "r") do fh
    for line in eachline(fh)
        sent = split(strip(line))
        sent = [w2i.(sent)]
        append!(sent[1],w2i("<s>"))
        append!(sent_arr,sent)
        end
    end
    return sent_arr
end


r (generic function with 1 method)

In [5]:
train = [(r(train_file))]
nwords = length(wdict)
test = [(r(test_file))]
S = w2i("<s>")
@test nwords == length(wdict)


Test Passed

In [8]:
length(train[1])

42068

In [6]:
train =reshape(train[1],(length(train[1])))
test = reshape(test[1],(length(test[1])))

3370-element Array{Any,1}:
 [2212, 397, 1130, 65, 832, 557, 809, 36, 1383, 2746, 65, 33, 2509, 1030, 25]                                   
 [27, 27, 1249, 8370, 120, 406, 2379, 3004, 384, 146  …  28, 29, 170, 76, 200, 468, 201, 27, 27, 25]            
 [653, 1194, 82, 3685, 43, 1119, 3682, 27, 946, 3947  …  36, 28, 199, 182, 1841, 153, 4110, 27, 2501, 25]       
 [49, 33, 102, 9085, 2063, 95, 885, 5164, 1053, 3685  …  182, 924, 7497, 153, 33, 529, 363, 120, 95, 25]        
 [9527, 1468, 1159, 70, 2443, 36, 102, 3935, 109, 27, 49, 1417, 9621, 160, 4782, 65, 4780, 33, 7630, 25]        
 [1964, 3685, 397, 7456, 27, 88, 33, 1159, 49, 27, 33, 765, 25]                                                 
 [316, 1486, 146, 9621, 160, 8662, 94, 3685, 149, 160  …  1622, 399, 557, 27, 1073, 27, 49, 5225, 27, 25]       
 [131, 2994, 553, 751, 36, 2032, 43, 556, 109, 3648  …  182, 802, 8370, 2191, 49, 3682, 120, 946, 4407, 25]     
 [88, 33, 8941, 43, 33, 1417, 393, 319, 364, 2775  …  679, 182, 36, 8

In [10]:
print(train[8])

[111, 112, 113, 114, 84, 80, 81, 36, 115, 82, 33, 116, 117, 118, 109, 119, 120, 102, 121, 122, 43, 123, 36, 124, 125, 65, 126, 102, 127, 65, 33, 128, 25]

In [7]:
EMBED_SIZE = 64
HIDDEN_SIZE = 128
WORDS_LOOKUP = param(nwords,EMBED_SIZE)

10000×64 Param{Array{Float32,2}}:
 -0.00518492    0.0063573     0.0060824    …  -0.0127958   -0.0100686 
  0.00618869   -0.0118947     0.0127541        0.00782637   0.0124991 
 -0.00276812    0.0115791    -0.00345769       0.0105092    0.00097859
  0.00672894    0.00553783   -0.00531367      -0.0115573   -0.00299659
  4.49189e-6    0.00359332    0.000904525     -0.00757715  -0.00897151
 -0.0055497    -0.00530192   -0.00752556   …   0.00859811   0.00145799
 -0.00427876   -0.00338799   -0.00846673       0.00633101   0.0113854 
 -0.00870852   -0.00419055   -0.00437413       0.0122682   -0.00619831
 -0.00217826    0.00824766   -0.00266837      -0.0104459   -0.00247628
  0.00670985   -0.00948394    0.0108614        0.00765955   0.00752631
 -0.00479205   -0.00935015    0.0066081    …   0.01234     -0.010391  
  0.00880401    0.0128416     0.00177239      -0.00631969   0.00116635
  0.0135329     0.00372931   -0.00113867      -0.00412305  -0.0116191 
  ⋮                                        

In [28]:
@doc param

```
param(array; atype)
param(dims...; init, atype)
param0(dims...; atype)
```

The first form returns `Param(atype(array))` where `atype=identity` is the default.

The second form Returns a randomly initialized `Param(atype(init(dims...)))`. By default, `init` is `xavier` and `atype` is `KnetArray{Float32}` if `gpu() >= 0`, `Array{Float32}` otherwise. 

The third form `param0` is an alias for `param(dims...; init=zeros)`.


In [8]:
rnn = RNN(EMBED_SIZE ,HIDDEN_SIZE,numLayers= 1 ,rnnType =:lstm)


LSTM(input=64,hidden=128)

In [9]:
W_exp = param(nwords,HIDDEN_SIZE)
b_exp = param(nwords)

10000-element Param{Array{Float32,1}}:
 -0.0003879292 
 -0.008033988  
 -0.0048559145 
  0.012462235  
 -0.010578476  
  0.008653249  
  0.00057372736
 -0.006008872  
 -0.0114769535 
  0.010327389  
 -0.008260677  
 -0.0071809604 
 -0.0034291758 
  ⋮            
 -0.0034405645 
  0.0026355356 
  0.005410014  
  0.009180684  
  0.0065267724 
  0.0063695824 
  0.013198854  
  0.0043724407 
  0.008379928  
 -0.0011470827 
 -0.0012697291 
  0.00159367   

In [9]:
@doc RNN

```
rnn = RNN(inputSize, hiddenSize; opts...)
rnn(x; batchSizes) => y
rnn.h, rnn.c  # hidden and cell states
```

`RNN` returns a callable RNN object `rnn`. Given a minibatch of sequences `x`, `rnn(x)` returns `y`, the hidden states of the final layer for each time step. `rnn.h` and `rnn.c` fields can be used to set the initial hidden states and read the final hidden states of all layers.  Note that the final time step of `y` always contains the final hidden state of the last layer, equivalent to `rnn.h` for a single layer network.

**Dimensions:** The input `x` can be 1, 2, or 3 dimensional and `y` will have the same number of dimensions as `x`. size(x)=(X,[B,T]) and size(y)=(H/2H,[B,T]) where X is inputSize, B is batchSize, T is seqLength, H is hiddenSize, 2H is for bidirectional RNNs. By default a 1-D `x` represents a single instance for a single time step, a 2-D `x` represents a single minibatch for a single time step, and a 3-D `x` represents a sequence of identically sized minibatches for multiple time steps. The output `y` gives the hidden state (of the final layer for multi-layer RNNs) for each time step. The fields `rnn.h` and `rnn.c` represent the hidden states of all layers in a single time step and have size (H,B,L/2L) where L is numLayers and 2L is for bidirectional RNNs.

**batchSizes:** If `batchSizes=nothing` (default), all sequences in a minibatch are assumed to be the same length. If `batchSizes` is an array of (non-increasing) integers, it gives us the batch size for each time step (allowing different sequences in the minibatch to have different lengths). In this case `x` will typically be 2-D with the second dimension representing variable size batches for time steps. If `batchSizes` is used, `sum(batchSizes)` should equal `length(x) ÷ size(x,1)`. When the batch size is different in every time step, hidden states will have size (H,B,L/2L) where B is always the size of the first (largest) minibatch.

**Hidden states:** The hidden and cell states are kept in `rnn.h` and `rnn.c` fields (the cell state is only used by LSTM). They can be initialized during construction using the `h` and `c` keyword arguments, or modified later by direct assignment. Valid values are `nothing` (default), `0`, or an array of the right type and size possibly wrapped in a `Param`. If the value is `nothing` the initial state is assumed to be zero and the final state is discarded keeping the value `nothing`. If the value is `0` the initial state is assumed to be zero and `0` is replaced by the final state on return. If the value is a valid state, it is used as the initial state and is replaced by the final state on return.

In a differentiation context the returned final hidden states will be wrapped in `Result` types. This is necessary if the same RNN object is to be called multiple times in a single iteration. Between iterations (i.e. after diff/update) the hidden states need to be unboxed with e.g. `rnn.h = value(rnn.h)` to prevent spurious dependencies. This happens automatically during the backward pass for GPU RNNs but needs to be done manually for CPU RNNs. See the [CharLM Tutorial](https://github.com/denizyuret/Knet.jl/blob/master/tutorial/80.charlm.ipynb) for an example.

**Keyword arguments for RNN:**

  * `h=nothing`: Initial hidden state.
  * `c=nothing`: Initial cell state.
  * `rnnType=:lstm` Type of RNN: One of :relu, :tanh, :lstm, :gru.
  * `numLayers=1`: Number of RNN layers.
  * `bidirectional=false`: Create a bidirectional RNN if `true`.
  * `dropout=0`: Dropout probability. Applied to input and between layers.
  * `skipInput=false`: Do not multiply the input with a matrix if `true`.
  * `dataType=Float32`: Data type to use for weights.
  * `algo=0`: Algorithm to use, see CUDNN docs for details.
  * `seed=0`: Random number seed for dropout. Uses `time()` if 0.
  * `winit=xavier`: Weight initialization method for matrices.
  * `binit=zeros`: Weight initialization method for bias vectors.
  * `finit=ones`: Weight initialization method for the bias of forget gates.
  * `usegpu=(gpu()>=0)`: GPU used by default if one exists.

**Formulas:** RNNs compute the output h[t] for a given iteration from the recurrent input h[t-1] and the previous layer input x[t] given matrices W, R and biases bW, bR from the following equations:

`:relu` and `:tanh`: Single gate RNN with activation function f:

```
h[t] = f(W * x[t] .+ R * h[t-1] .+ bW .+ bR)
```

`:gru`: Gated recurrent unit:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
r[t] = sigm(Wr * x[t] .+ Rr * h[t-1] .+ bWr .+ bRr) # reset gate
n[t] = tanh(Wn * x[t] .+ r[t] .* (Rn * h[t-1] .+ bRn) .+ bWn) # new gate
h[t] = (1 - i[t]) .* n[t] .+ i[t] .* h[t-1]
```

`:lstm`: Long short term memory unit with no peephole connections:

```
i[t] = sigm(Wi * x[t] .+ Ri * h[t-1] .+ bWi .+ bRi) # input gate
f[t] = sigm(Wf * x[t] .+ Rf * h[t-1] .+ bWf .+ bRf) # forget gate
o[t] = sigm(Wo * x[t] .+ Ro * h[t-1] .+ bWo .+ bRo) # output gate
n[t] = tanh(Wn * x[t] .+ Rn * h[t-1] .+ bWn .+ bRn) # new gate
c[t] = f[t] .* c[t-1] .+ i[t] .* n[t]               # cell output
h[t] = o[t] .* tanh(c[t])
```


In [14]:
S

25

In [27]:
WORDS_LOOKUP

10000×64 Param{Array{Float32,2}}:
  0.00196528   -0.00938462    0.00567422   …  -0.0136769     0.0138187  
 -0.00604959    0.0109012    -0.0128035        0.010893      0.0120626  
  0.0105923    -0.013568      0.00961081       0.0133092     0.0118643  
 -0.000280146  -0.0128946    -0.0073938        0.00343786   -0.00113664 
 -0.00166148   -0.0132606    -0.000856147      0.00991218    0.000431897
  0.0017868    -0.0127415    -0.0125457    …   0.000639681   0.013581   
  0.00179157    0.0112696     0.0108029        0.00113457    0.00278058 
  0.00790284    0.000665226   0.0104059       -0.00883937    0.0131455  
 -0.0036514     0.00264432   -0.00520426      -0.0134825     0.00667357 
  0.0140459     0.00274444    0.00970449      -0.00375891    0.0129993  
  0.00518108   -0.011663      0.00484968   …   0.00598954    0.011242   
  0.00400151    0.0112102    -0.00795942      -0.00264307    0.0102423  
 -0.0125582     0.00299876    0.00231         -0.0121602    -0.0105763  
  ⋮              

In [16]:
rnn.h=0
rnn.c =0


0

In [10]:
y =rnn(WORDS_LOOKUP[S,:])

128-element Array{Float32,1}:
 -0.00090743304
  0.00048159395
 -0.0010099952 
 -0.00031180202
 -0.00046426308
  0.0015844299 
 -0.00080187654
  3.3149012e-5 
 -0.0001679539 
 -0.0007280562 
 -0.0011277992 
 -0.00033800787
  0.00037492733
  ⋮            
 -5.221445e-5  
  0.00027401824
 -0.00016411903
 -0.0006456478 
  0.00054729934
 -0.0005219595 
  0.0006447841 
 -0.00089673116
  0.0003028171 
 -0.0001353141 
  0.0004410099 
 -0.00057321205

In [11]:
function calc_lm_loss(sent)
    tot_words = length(sent)
    rnn.h= 0 
    rnn.c = 0
    y = rnn(WORDS_LOOKUP[S,:])
    losses=[]
    for wid in sent
        score = W_exp * y+b_exp
        loss,_ = nll(score,[wid],average= false)
        push!(losses,loss)
        wemb = WORDS_LOOKUP[wid,:]
        y = rnn(wemb)
    end
    return sum(losses),tot_words
    
end

calc_lm_loss (generic function with 1 method)

In [20]:
@doc nll

```
nll(scores, answers; dims=1, average=true)
```

Given an unnormalized `scores` matrix and an `Integer` array of correct `answers`, return the negative log likelihood. The `scores` matrix should have size (classes,instances) if `dims=1` or (instances,classes) if `dims=2`. `answers[i]` should be in `1:classes` to indicate the correct class for instance i, or 0 to skip instance i. The return value is `(total/count)` if `average=true` and `(total,count)` if `average=false` where `count` is the number of instances not skipped and `total` is their total negative log likelihood.

```
nll(model, data; dims=1, average=true, o...)
```

Compute negative log likelihood `nll(model(x; o...), y; dims)` for `(x,y)` in `data` and return `(total/count)` if `average=true` or `(total,count)` if `average=false`.


In [22]:
rand()

0.7684476751965699

In [14]:
arr = [1,2,3,4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [16]:
calc_lm_loss(arr)

(36.842243f0, 4)

In [24]:
train_order = (1:length(train))

1:42068

In [25]:
length(train_order)

42068

In [10]:
print("startup time: %r" % (time() - start))


UndefVarError: UndefVarError: start not defined

In [26]:
i = all_time = dev_time = all_tagged = this_words = this_loss = 0


0

In [18]:
for ITER in 1:100
    print(ITER)
end

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100

In [23]:
shuffle(train_order)

100-element Array{Int64,1}:
 27
 59
 14
 66
 77
 96
 15
 31
 25
 12
 42
 86
 50
  ⋮
 67
 69
 65
  2
  6
 60
 28
  7
 81
 85
  8
 90

In [44]:
test[2]

BoundsError: BoundsError: attempt to access 1-element Array{Array{Any,1},1} at index [2]

In [53]:
exp(2)

7.38905609893065

In [66]:
a =5/2
print("a equals"*string(a)*"asd")

a equals2.5asd

In [67]:
#print("startup time: %r" % (time.time() - start))
# Perform training
#start = time.time()
i = all_time = dev_time = all_tagged = this_words = this_loss = 0
for ITER in 1:100
    shuffle(train_order)
    for sid in train_order
        i += 1
        if i % 1 == 0
            #trainer.status()
            print(this_loss / this_words)
            all_tagged += this_words
            this_loss = this_words = 0
           # all_time = time.time() - start
        end
        if i % 2 == 0
            #dev_start = time.time()
            dev_loss = dev_words = 0
            for sent in test
                loss_exp, mb_words = calc_lm_loss(sent)
                dev_loss += loss_exp
                dev_words += mb_words
            end
           # dev_time += time.time() - dev_start
            #train_time = time.time() - start - dev_time
            nl = dev_loss/dev_words
            ppl = exp(dev_loss/dev_words)
            print("NLL: "*nl*" ppl : "*ppl)
        # train on the minibatch
        loss_exp, mb_words = calc_lm_loss(train[sid])
        this_loss += loss_exp
        this_words += mb_words
        loss_exp.backward()
        trainer.update()
        end
    end
    print("epoch %r finished" % ITER)
    trainer.update_epoch(1.0)
end

NaNNaN

MethodError: MethodError: no method matching *(::String, ::Float32)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  *(!Matched::Bool, ::T<:AbstractFloat) where T<:AbstractFloat at bool.jl:118
  *(!Matched::Float32, ::Float32) at float.jl:398
  ...